1. Load the CSV file into a DataFrame.
2. Parse the "Radiomics" column, as it contains JSON data.
3. Remove columns with the same values across all rows.

In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os

# create random seed for reproducibility
ran_seed = 42

# Load the data from DF_Radiomics_noduls_with_diagnose.csv
file_path = "DF_Radiomics_noduls_with_diagnose.csv"
data = pd.read_csv(file_path)


# Convert the 'Labels' column to an integer
data['Labels'] = data['Labels'].astype(int)

# drop all rows where the label == 0
data = data[data.Labels != 0]

# Parse the JSON in the 'Radiomics' column
data['Radiomics'] = data['Radiomics'].apply(json.loads)

# Convert the 'Radiomics' column into separate columns
radiomics_data = pd.json_normalize(data['Radiomics'])


# Drop the original 'Radiomics' column
data = data.drop('Radiomics', axis=1)


# Reset the indices of both DataFrames
data = data.reset_index(drop=True)
radiomics_data = radiomics_data.reset_index(drop=True)

# Combine the data with the new radiomics columns
data = pd.concat([data, radiomics_data], axis=1)

# Remove columns with the same value across all rows
data = data.loc[:, (data != data.iloc[0]).any()]

#remove columns with all NaN values
data = data.dropna(axis=1, how='all')

print(data.shape)


(309, 103)


In [3]:
data.head()

,Patient,Node,Labels,diagnostics_Image-original_Hash,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,...,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis
0,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,0506d1d0d6522eddd1640c8ea75c2fc5a9266270,...,7.355556,60.706173,469.644444,23.444444,16578.377778,0.053875,0.021012,0.488461,152.929922,0.019809
1,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,9d7da356d43e2f7ad7f374f6c193e97f6088d7c7,...,7.467153,72.801002,471.051095,17.496350,13573.328467,0.110650,0.024328,0.494688,165.356306,0.010062
2,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,c0a43747a23d26b107e21614525f2fd8870ffefc,...,7.685185,43.527006,277.787037,20.370370,9310.490741,0.084481,0.031811,0.463956,84.174037,0.027819
3,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,72a09dc3f5d5d146b13402b8ef109422cc3f38a5,...,6.780220,35.367709,229.219780,18.780220,7065.923077,0.084783,0.026368,0.465301,67.725183,0.021973
4,LIDC-IDRI-0072,Node_N1,1,54705f26f9320581c90452445aa820fe9630d5e9,"[0.732422, 0.732422, 1.25]","[512, 512, 305]",-871.936330,-3024.0,3071.0,05efcefff38c73903c3d7839bb987a49176f6068,...,629.334146,45.147393,1253.131545,28.918031,43475.541623,0.020967,0.001319,0.262518,254.476429,0.000632


In [4]:
#remove hash columns
data = data.drop(['diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'], axis=1)

# ok looks like all the objeckt columns except of "Patient" & "Node" are in this form [0.683594, 0.683594, 1.25] which is a list of multiple floats
# exploade them into multiple columns

object_columns = data.select_dtypes(include=['object']).columns.tolist()

# Remove 'Patient' and 'Node' from the list
object_columns.remove('Patient')
object_columns.remove('Node')

# Explode the lists in each object column into multiple columns
for column in object_columns:
    # Convert each list to a Series and expand it into multiple columns
    expanded_columns = data[column].apply(pd.Series)
    
    # Rename the expanded columns to have the original column name as a prefix
    expanded_columns = expanded_columns.rename(columns=lambda x: f"{column}_{x}")
    
    # Drop the original column from the DataFrame
    data = data.drop(column, axis=1)
    
    # Concatenate the expanded columns to the DataFrame
    data = pd.concat([data, expanded_columns], axis=1)

In [5]:
# Create a stratified split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'], random_state=ran_seed)

# if the files already exist, skip this step
if os.path.isfile('DF_Radiomics_noduls_with_diagnose_train_data.csv') and os.path.isfile('DF_Radiomics_noduls_with_diagnose_test_data.csv'):
    print("Files already exist, skipping this step")
else:
    # Save the data to CSV files
    train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data.csv', index=False)
    test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data.csv', index=False)

Files already exist, skipping this step


In [6]:
print("Train data:", train_data.shape)
print("Test data:", test_data.shape)

Train data: (247, 118)
Test data: (62, 118)


In [7]:
train_data.head()

,Patient,Node,Labels,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,...,diagnostics_Mask-original_BoundingBox_2,diagnostics_Mask-original_BoundingBox_3,diagnostics_Mask-original_BoundingBox_4,diagnostics_Mask-original_BoundingBox_5,diagnostics_Mask-original_CenterOfMassIndex_0,diagnostics_Mask-original_CenterOfMassIndex_1,diagnostics_Mask-original_CenterOfMassIndex_2,diagnostics_Mask-original_CenterOfMass_0,diagnostics_Mask-original_CenterOfMass_1,diagnostics_Mask-original_CenterOfMass_2
23,LIDC-IDRI-0137,Node_N1,3,-671.885608,-2048.0,3071.0,26,1,175.5,850.5,...,30,4,6,2,332.692308,389.538462,30.307692,53.215868,83.626926,-321.730769
263,LIDC-IDRI-0377,Node_N1,2,-882.321409,-3024.0,3071.0,2402,1,-307.0,61.0,...,169,29,24,9,382.402998,308.854288,173.039550,92.739302,28.898399,-68.460564
44,LIDC-IDRI-0167,Node_N1,1,-664.766231,-2048.0,3071.0,56,1,-444.5,-66.5,...,50,6,9,2,70.267857,174.964286,50.321429,-136.237780,-53.812866,-234.696429
219,LIDC-IDRI-0272,Node_N1,3,-824.358062,-2048.0,3071.0,51,1,-447.0,102.0,...,81,6,7,2,209.313725,390.941176,81.568627,-47.673652,80.722794,-109.078431
143,LIDC-IDRI-0234,Node_N1,1,-708.012378,-2048.0,3029.0,251,1,-569.0,82.0,...,41,11,14,3,367.756972,310.848606,41.689243,65.179121,43.765426,-236.276892


Scaling the data

In [8]:
# if DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv and DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv already exist, skip this step
# otherwise scale the data and save it to CSV files
if os.path.isfile('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv') and os.path.isfile('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv'):
    print("Scaled data already exists")
else:
    from sklearn.preprocessing import StandardScaler

    # Get all column names
    all_columns = train_data.columns.tolist()

    # Exclude the first three columns
    features = all_columns[3:]

    # Create a stratified split
    train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'])

    # Create a scaler
    scaler = StandardScaler()

    # Fit the scaler on the training data and transform both training and test data
    train_data[features] = scaler.fit_transform(train_data[features])
    test_data[features] = scaler.transform(test_data[features])

    # Save the data to CSV files
    train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv', index=False)
    test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv', index=False)


Scaled data already exists


# Fully Connected Neural Network

In [9]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming 'data' is your pandas DataFrame
# Ensure the DataFrame only contains numeric values
data = pd.read_csv('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv')
#drop patient and node columns
data = data.drop(['Patient', 'Node'], axis=1)
# TODO maybe add the columns later to see if it helps

Trainset

In [10]:
# Split data into features and labels
X = data.drop('Labels', axis=1).values
y = data['Labels'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X).float()
y_tensor = torch.tensor(y).float()

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor, random_state=ran_seed)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [11]:
print("Train data:", train_dataset.tensors[0].shape)

Train data: torch.Size([197, 115])


In [12]:
# Model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size, hidden_size*2),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size*2, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Hyperparameters
input_size = train_dataset.tensors[0].shape[1]  # Get the number of features from your dataset
hidden_size = input_size*2  # You can tune this
output_size = 4   # 3 labels 
learning_rate = 0.001
batch_size = 32
epochs = 50  # Adjust based on your runtime requirement
early_stopping_factor = 10
clip_value = 1  # for gradient clipping

# Initialize model, loss function, and optimizer
model = FCNN(input_size, hidden_size, output_size).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 regularization

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [13]:
#check if cuda is available, print the gpu model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    # Move model to the device
    model = model.to(device)

NVIDIA GeForce RTX 3090


In [14]:
# Initialize best loss to infinity for comparison in the first epoch
best_loss = float('inf')

# Patience counter
patience_counter = 0

# Patience limit
patience_limit = 5

# Training loop
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for inputs, targets in train_loader:
    
        # Move inputs and targets to the device
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        targets = targets.long()

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Average epoch loss
    epoch_loss = epoch_loss / len(train_loader)

    # If the training loss has improved, save the model and reset the patience counter
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        # If the training loss has not improved, increment the patience counter
        patience_counter += 1
        if patience_counter >= patience_limit:
            print(f"Early stopping at epoch {epoch+1}/{epochs}, best loss: {best_loss}")
            break

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}")

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

Epoch 1/50, Loss: 1.3304967199053084
Epoch 2/50, Loss: 1.1353100197655814
Epoch 3/50, Loss: 1.0851800782339913
Epoch 4/50, Loss: 0.9991901431764875
Epoch 5/50, Loss: 0.87434093441282
Epoch 6/50, Loss: 0.7975443005561829
Epoch 7/50, Loss: 0.9110166685921806
Epoch 8/50, Loss: 0.6931230000087193
Epoch 9/50, Loss: 0.662830039858818
Epoch 10/50, Loss: 0.5734744497707912
Epoch 11/50, Loss: 0.4815047766481127
Epoch 12/50, Loss: 0.4496862547738211
Epoch 13/50, Loss: 0.3978220011506762
Epoch 14/50, Loss: 0.3232405888182776
Epoch 15/50, Loss: 0.34018705572400776
Epoch 16/50, Loss: 0.31875818754945484
Epoch 17/50, Loss: 0.3030732146331242
Epoch 18/50, Loss: 0.24767346041543142
Epoch 19/50, Loss: 0.18171399193150656
Epoch 20/50, Loss: 0.22024099635226385
Epoch 21/50, Loss: 0.21861376400504792
Epoch 22/50, Loss: 0.10599877632090024
Epoch 23/50, Loss: 0.10694949488554682
Epoch 24/50, Loss: 0.08373230908598218
Epoch 25/50, Loss: 0.06051187333650887
Epoch 26/50, Loss: 0.07139431046588081
Epoch 27/50, 

<All keys matched successfully>

# Evaluate

Since Confusion Matrix is 3x3 calculate Sensitivity & Specificity for each class by considering that class as the positive class and the other two as the negative class.

Sensitivity, also known as the true positive rate (TPR), measures the proportion of actual positives that are correctly identified as such. In other words, it measures the ability of the model to correctly identify positive instances.

Specificity, on the other hand, measures the proportion of actual negatives that are correctly identified as such. It measures the ability of the model to correctly identify negative instances.

The false positive rate (FPR) is the complement of specificity. It measures the proportion of actual negatives that are incorrectly identified as positives. In other words, it measures the rate at which the model makes false alarms.

Here's how they relate:

- TPR = Sensitivity = TP / (TP + FN)
- FPR = 1 - Specificity = FP / (FP + TN)
- Specificity = TN / (TN + FP)

Where:
- TP = True Positives
- FN = False Negatives
- FP = False Positives
- TN = True Negatives

In [15]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient calculations
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability

            total_predictions += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = correct_predictions / total_predictions

    # Calculate confusion matrix
    cm = confusion_matrix(all_targets, all_predictions)

    # Calculate sensitivity and specificity for each class
    sensitivity = np.diag(cm) / np.sum(cm, axis = 1)
    specificity = (np.sum(cm) - np.sum(cm, axis = 0) - np.sum(cm, axis = 1) + np.diag(cm)) / (np.sum(cm) - np.sum(cm, axis = 0))

    return accuracy, sensitivity, specificity

## Testset

In [16]:
# Use the function
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
accuracy, sensitivity, specificity = evaluate(model, test_loader, device)
print(f'Accuracy: {accuracy * 100}%')
for i, (sens, spec) in enumerate(zip(sensitivity, specificity)):
    print(f'Class {i}: Sensitivity: {sens * 100}%, Specificity: {spec * 100}%')

Accuracy: 78.0%
Class 0: Sensitivity: 75.0%, Specificity: 89.1891891891892%
Class 1: Sensitivity: 76.92307692307693%, Specificity: 90.9090909090909%
Class 2: Sensitivity: 80.95238095238095%, Specificity: 86.66666666666667%


## Validation Set

In [17]:
validation_data = pd.read_csv('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv')

#create the tensor dataset
X_test = validation_data.drop(['Patient', 'Node', 'Labels'], axis=1).values
y_test = validation_data['Labels'].values
validation_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).float())

# Use the function
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
accuracy, sensitivity, specificity = evaluate(model, validation_loader, device)
print(f'Accuracy: {accuracy * 100}%')
for i, (sens, spec) in enumerate(zip(sensitivity, specificity)):
    print(f'Class {i}: Sensitivity: {sens * 100}%, Specificity: {spec * 100}%')

Accuracy: 88.70967741935483%
Class 0: Sensitivity: 95.0%, Specificity: 97.5609756097561%
Class 1: Sensitivity: 87.5%, Specificity: 95.55555555555556%
Class 2: Sensitivity: 84.61538461538461%, Specificity: 89.47368421052632%


# Gridsearch

In [49]:
# Hyperparameters
input_size = train_dataset.tensors[0].shape[1]  # Get the number of features from your dataset
output_size = 4   # 3 labels 
batch_size = 32
epochs = 50  # Adjust based on your runtime requirement
early_stopping_factor = 10
clip_value = 1  # for gradient clipping

# Hyperparameters to tune
hidden_sizes = [input_size*4, input_size*5, input_size*6]
learning_rates = [0.0001, 0.001, 0.01]
weight_decays = [0, 1e-5, 1e-4]  # L2 regularization

# Load validation data
validation_data = pd.read_csv('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv')

# Create the tensor dataset
X_test = validation_data.drop(['Patient', 'Node', 'Labels'], axis=1).values
y_test = validation_data['Labels'].values
validation_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).float())

# Use the function
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

# Initialize DataFrame to store results
results = pd.DataFrame(columns=["epoch", "epoch_loss", "best_loss", "hidden_size", "learning_rate", "weight_decay", "accuracy", "sensitivity", "specificity"])

# Grid search
for hidden_size in hidden_sizes:
    for learning_rate in learning_rates:
        for weight_decay in weight_decays:
            # Initialize model, loss function, and optimizer
            model = FCNN(input_size, hidden_size, output_size).cuda()
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

            # DataLoader
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
                        # Initialize best loss to infinity for comparison in the first epoch
            best_loss = float('inf')

            # Patience counter
            patience_counter = 0

            # Patience limit
            patience_limit = 5
            
            # Training loop (as before)...
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0
                for inputs, targets in train_loader:
                
                    # Move inputs and targets to the device
                    inputs = inputs.to(device)
                    targets = targets.to(device)
                    
                    targets = targets.long()

                    # Zero the gradients
                    optimizer.zero_grad()

                    # Forward pass
                    outputs = model(inputs)

                    # Compute loss
                    loss = criterion(outputs, targets)

                    # Backward pass and optimize
                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item()

                # Average epoch loss
                epoch_loss = epoch_loss / len(train_loader)

                # If the training loss has improved, save the model and reset the patience counter
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    patience_counter = 0
                    #torch.save(model.state_dict(), 'best_model.pth')
                    #change to realtive path according to the hidden size, learning rate and weight decay
                    torch.save(model.state_dict(), f'../models/best_model_{hidden_size}_{str(learning_rate).replace(".", "_")}_{weight_decay}.pth')

                    
                else:
                    # If the training loss has not improved, increment the patience counter
                    patience_counter += 1
                    if patience_counter >= patience_limit:
                        print(f"Early stopping at epoch {epoch+1}/{epochs}, best loss: {best_loss}")
                        break

                print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}")

            # Load the best model
            model.load_state_dict(torch.load(f'../models/best_model_{hidden_size}_{str(learning_rate).replace(".", "_")}_{weight_decay}.pth'))

            # Evaluate the model
            accuracy, sensitivity, specificity = evaluate(model, validation_loader, device)

            # Write results to DataFrame
            # Check if results is a DataFrame with concat
            results = pd.concat([results, pd.DataFrame([[epoch, epoch_loss, best_loss, hidden_size, learning_rate, weight_decay, accuracy, sensitivity, specificity]], columns=results.columns)])


# Print the results
print(results)

Epoch 1/50, Loss: 1.3779354436056954
Epoch 2/50, Loss: 1.3421578407287598
Epoch 3/50, Loss: 1.3160295145852225
Epoch 4/50, Loss: 1.2681296042033605
Epoch 5/50, Loss: 1.232401932988848
Epoch 6/50, Loss: 1.1976555926459176
Epoch 7/50, Loss: 1.1565898997443063
Epoch 8/50, Loss: 1.1289265666689192
Epoch 9/50, Loss: 1.1114522559302193
Epoch 10/50, Loss: 1.0388939210346766
Epoch 11/50, Loss: 1.021112757069724
Epoch 12/50, Loss: 1.026507556438446
Epoch 13/50, Loss: 0.9901436652456012
Epoch 14/50, Loss: 0.9268379552023751
Epoch 15/50, Loss: 0.9635300976889474
Epoch 16/50, Loss: 0.8990778071539742
Epoch 17/50, Loss: 0.8695678796086993
Epoch 18/50, Loss: 0.8551375355039325
Epoch 19/50, Loss: 0.7835171478135246
Epoch 20/50, Loss: 0.7730300256184169
Epoch 21/50, Loss: 0.7588641047477722
Epoch 22/50, Loss: 0.750460548060281
Epoch 23/50, Loss: 0.7231575506074088
Epoch 24/50, Loss: 0.6997528076171875
Epoch 25/50, Loss: 0.6496872391019549
Epoch 26/50, Loss: 0.7131269148417881
Epoch 27/50, Loss: 0.6161

/tmp/ipykernel_615/3107658289.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([[epoch, epoch_loss, best_loss, hidden_size, learning_rate, weight_decay, accuracy, sensitivity, specificity]], columns=results.columns)])


Epoch 11/50, Loss: 1.0272281680788313
Epoch 12/50, Loss: 0.9932590041841779
Epoch 13/50, Loss: 0.9979591029030936
Epoch 14/50, Loss: 0.9793221269335065
Epoch 15/50, Loss: 0.9294391615050179
Epoch 16/50, Loss: 0.9041635394096375
Epoch 17/50, Loss: 0.8433543017932347
Epoch 18/50, Loss: 0.7912102511950901
Epoch 19/50, Loss: 0.8612799559320722
Epoch 20/50, Loss: 0.8001096163477216
Epoch 21/50, Loss: 0.7574533479554313
Epoch 22/50, Loss: 0.8634302360670907
Epoch 23/50, Loss: 0.7931511998176575
Epoch 24/50, Loss: 0.6860793488366264
Epoch 25/50, Loss: 0.6872975826263428
Epoch 26/50, Loss: 0.6372499508517129
Epoch 27/50, Loss: 0.6614081135817936
Epoch 28/50, Loss: 0.6357598773070744
Epoch 29/50, Loss: 0.5535514567579541
Epoch 30/50, Loss: 0.6209481869425092
Epoch 31/50, Loss: 0.5285731085709163
Epoch 32/50, Loss: 0.5344104255948748
Epoch 33/50, Loss: 0.5277976053101676
Epoch 34/50, Loss: 0.48495069997651236
Epoch 35/50, Loss: 0.47730797954968046
Epoch 36/50, Loss: 0.4837290474346706
Epoch 37/5

/tmp/ipykernel_615/4158225697.py:33: RuntimeWarning: invalid value encountered in divide
  specificity = (np.sum(cm) - np.sum(cm, axis = 0) - np.sum(cm, axis = 1) + np.diag(cm)) / (np.sum(cm) - np.sum(cm, axis = 0))


Epoch 6/50, Loss: 1.0870654668126787
Epoch 7/50, Loss: 1.0764166968209403
Epoch 8/50, Loss: 1.100658995764596
Epoch 9/50, Loss: 1.4247029764311654
Epoch 10/50, Loss: 1.0745293242590768
Epoch 11/50, Loss: 1.1438286985669817
Epoch 12/50, Loss: 1.112262521471296
Epoch 13/50, Loss: 1.094802669116429
Epoch 14/50, Loss: 1.1010198933737618
Early stopping at epoch 15/50, best loss: 1.0745293242590768


/tmp/ipykernel_615/4158225697.py:33: RuntimeWarning: invalid value encountered in divide
  specificity = (np.sum(cm) - np.sum(cm, axis = 0) - np.sum(cm, axis = 1) + np.diag(cm)) / (np.sum(cm) - np.sum(cm, axis = 0))


Epoch 1/50, Loss: 2.5364006076540266
Epoch 2/50, Loss: 1.777518289429801
Epoch 3/50, Loss: 1.357111896787371
Epoch 4/50, Loss: 1.4341205188206263
Epoch 5/50, Loss: 1.2256047895976476
Epoch 6/50, Loss: 1.2953356845038277
Epoch 7/50, Loss: 1.0882683311189925
Epoch 8/50, Loss: 0.9882844090461731
Epoch 9/50, Loss: 1.1003705518586295
Epoch 10/50, Loss: 1.0888277207102095
Epoch 11/50, Loss: 1.026561668940953
Epoch 12/50, Loss: 1.1490522537912642
Early stopping at epoch 13/50, best loss: 0.9882844090461731
  epoch  epoch_loss  best_loss hidden_size  learning_rate weight_decay  \
0    49    0.205147   0.205147         460         0.0001            0   
0    49    0.269517   0.231874         460         0.0001      0.00001   
0    49    0.225831   0.221439         460         0.0001       0.0001   
0    29    0.074579   0.027501         460         0.0010            0   
0    40    0.001394   0.000838         460         0.0010      0.00001   
0    26    0.030852   0.021105         460         

In [50]:
#calculate a score from accuracy, sensitivity and specificity wher sensitivity and specificity are a list of 3 values
results['score'] = results['accuracy'] + results['sensitivity'].apply(lambda x: sum(x)) + results['specificity'].apply(lambda x: sum(x))
#sort by score
results = results.sort_values(by=['score'], ascending=False)
results

,epoch,epoch_loss,best_loss,hidden_size,learning_rate,weight_decay,accuracy,sensitivity,specificity,score
0,28,0.105441,0.007929,690,0.0010,0,0.870968,"[0.85, 0.875, 0.8846153846153846]","[0.9302325581395349, 0.9555555555555556, 0.916...",6.283038
0,40,0.001394,0.000838,460,0.0010,0.00001,0.854839,"[0.95, 0.875, 0.7692307692307693]","[0.972972972972973, 0.9574468085106383, 0.85]",6.229489
0,26,0.032074,0.009340,690,0.0010,0.00001,0.854839,"[0.9, 0.8125, 0.8461538461538461]","[0.9512195121951219, 0.9347826086956522, 0.891...",6.191387
0,29,0.074579,0.027501,460,0.0010,0,0.838710,"[0.95, 0.8125, 0.7692307692307693]","[0.972972972972973, 0.9375, 0.8461538461538461]",6.127067
0,20,0.091267,0.034236,690,0.0010,0.0001,0.838710,"[0.95, 0.8125, 0.7692307692307693]","[0.9736842105263158, 0.9361702127659575, 0.846...",6.126449
0,49,0.060604,0.060604,690,0.0001,0.0001,0.838710,"[0.95, 0.8125, 0.7692307692307693]","[0.9743589743589743, 0.9347826086956522, 0.846...",6.125736
0,30,0.159257,0.006020,575,0.0010,0.0001,0.822581,"[0.95, 0.8125, 0.7307692307692307]","[0.972972972972973, 0.9361702127659575, 0.825]",6.049993
0,49,0.205147,0.205147,460,0.0001,0,0.822581,"[0.9, 0.8125, 0.7692307692307693]","[0.9473684210526315, 0.9375, 0.8421052631578947]",6.031285
0,42,0.172360,0.006611,575,0.0010,0.00001,0.822581,"[0.85, 0.8125, 0.8076923076923077]","[0.926829268292683, 0.9347826086956522, 0.8648...",6.019250
0,49,0.065157,0.065157,690,0.0001,0,0.806452,"[0.95, 0.8125, 0.6923076923076923]","[0.972972972972973, 0.9347826086956522, 0.8048...",5.973893
